# Final capstone project for Coursera IBM Data science certification

## Introduction:

As part of the IBM Data science certification on Coursera, we were asked to perform analysis and clustering based on location data from FourSquare. In the earlier classes, we had done segmentation of New York and Toronto cities. I decided to extend the Toronto analysis further for the project. As we are in the midst of a COVID-19 pandemic, I planned to analyse the Toronto location data along with the COVID-19 reported cases in Toronto, to identify clusters of locations which might be risky for people to go to. This could help people avoid venues with higher concentration of active cases. This can further be segmented by Age groups of people who have been infected and the kind of venues which are located close by. 

## Business problem:

As we move into a Post-COVID world, and the cities relaxing the restrictions, it will be important for people to know the high risk areas with active cases and take necessary precaution or avoid the venues in neighborhoods with a higher case count. 

## Data:

I will be leveraging 3 data sources for this project
1. The neighborhoods of Toronto as per the Toronto Open data portal. This data set has the list of 140 neighborhoods in Toronto along with latitude & longitude, and also the geometry of the neighborhood. 
2. Foursquare data for the most popular public places in certain areas. This data set will be based on the output of the API from Foursquare to fetch the location and venue details. 
3. COVID-19 cases reported in each neighborhood from the Toronto Open data portal. This dataset is refreshed daily by the Toronto portal. This data set has details of each case reported, the age group of the person, neighborhood in Toronto, gender, and status pf the case (Active, Resolved, Fatal). For the purpose of this project, I will use only active cases for analysis. 

By combining these three sources of data, we can cluster the venues based on the density of COVID cases.


In [1]:
import pandas as pd
import requests
 
# Get the dataset metadata by passing package_id to the package_search endpoint
# For example, to retrieve the metadata for this dataset:
 
url = "https://ckan0.cf.opendata.inter.prod-toronto.ca/api/3/action/package_show"
params = { "id": "64b54586-6180-4485-83eb-81e8fae3b8fe"}
package = requests.get(url, params = params).json()
print(package["result"])
 
# Get the data by passing the resource_id to the datastore_search endpoint
# See https://docs.ckan.org/en/latest/maintaining/datastore.html for detailed parameters options
# For example, to retrieve the data content for the first resource in the datastore:
 
for idx, resource in enumerate(package["result"]["resources"]):
    if resource["datastore_active"]:
        url = "https://ckan0.cf.opendata.inter.prod-toronto.ca/api/3/action/datastore_search"
        p = { "id": resource["id"] }
        data = requests.get(url, params = p).json()
        df = pd.DataFrame(data["result"]["records"])
        break
df

{'license_title': 'Open Government Licence – Toronto', 'owner_unit': 'Communicable Disease Surveillance Unit', 'relationships_as_object': [], 'topics': 'Health', 'owner_email': 'cdsu@toronto.ca', 'excerpt': 'Line-listed report of COVID-19 cases among Toronto residents, including demographic, severity, geographical, and epidemiological variables.', 'private': False, 'owner_division': 'Toronto Public Health', 'num_tags': 7, 'id': '64b54586-6180-4485-83eb-81e8fae3b8fe', 'metadata_created': '2020-07-10T14:00:54.343339', 'refresh_rate': 'Weekly', 'title': 'COVID-19 Cases in Toronto', 'license_url': 'https://open.toronto.ca/open-data-license/', 'state': 'active', 'information_url': 'https://www.toronto.ca/home/covid-19/covid-19-latest-city-of-toronto-news/covid-19-status-of-cases-in-toronto/', 'license_id': 'open-government-licence-toronto', 'type': 'dataset', 'resources': [{'cache_last_updated': None, 'package_id': '64b54586-6180-4485-83eb-81e8fae3b8fe', 'datastore_active': True, 'id': 'e5b

,_id,Outbreak Associated,Age Group,Neighbourhood Name,FSA,Source of Infection,Classification,Episode Date,Reported Date,Client Gender,Outcome,Currently Hospitalized,Currently in ICU,Currently Intubated,Ever Hospitalized,Ever in ICU,Ever Intubated
0,44069,Sporadic,70-79,NaN,NaN,Close contact,PROBABLE,2020-06-01,2020-06-01,MALE,RESOLVED,No,No,No,No,No,No
1,44070,Sporadic,40-49,NaN,NaN,Community,CONFIRMED,2020-05-05,2020-06-02,MALE,RESOLVED,No,No,No,No,No,No
2,44071,Sporadic,40-49,NaN,NaN,Close contact,CONFIRMED,2020-05-24,2020-05-25,MALE,RESOLVED,No,No,No,Yes,No,No
3,44072,Sporadic,19 and younger,NaN,NaN,Close contact,PROBABLE,2020-05-28,2020-05-25,MALE,RESOLVED,No,No,No,No,No,No
4,44073,Sporadic,60-69,NaN,NaN,Community,CONFIRMED,2020-06-04,2020-06-27,MALE,RESOLVED,No,No,No,No,No,No
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,40270,Sporadic,20-29,Alderwood,M8W,Close contact,CONFIRMED,2020-04-20,2020-04-21,FEMALE,RESOLVED,No,No,No,No,No,No
96,40271,Sporadic,90+,Alderwood,M8W,Healthcare,CONFIRMED,2020-04-03,2020-04-19,FEMALE,FATAL,No,No,No,Yes,No,No
97,40272,Sporadic,30-39,Alderwood,M8W,Community,CONFIRMED,2020-05-02,2020-05-13,FEMALE,RESOLVED,No,No,No,No,No,No
98,40273,Sporadic,60-69,Alderwood,M8W,Close contact,CONFIRMED,2020-05-05,2020-05-16,FEMALE,RESOLVED,No,No,No,No,No,No
